bismillah

stop ketika udah kena jangkauan, limit sampe 5 aja


In [1]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.animation import FuncAnimation
from skyfield.api import EarthSatellite, load, wgs84
from datetime import datetime, timedelta, timezone
from shapely.geometry import Polygon, Point # Tambah Point
from shapely.ops import unary_union
from pyproj import Geod
import csv, os

# ---------- TLE ISS (ZARYA) ----------
tle_lines = [
    "EXPLORER 22",
    "1 00899U 64064A   25276.49600160  .00000579  00000-0  49569-3 0  9991",
    "2 00899  79.6909  47.8810 0120383 146.1380 214.7533 13.82947257 69497"
]
# -------------------------------------

update_interval_ms = 200     # ms per frame
speedup = 127                # percepatan waktu simulasi
spotbeam_width_km = 2000     # lebar strip coverage tegak lurus lintasan (km)

# Inisialisasi skyfield
ts = load.timescale()
sat = EarthSatellite(tle_lines[1], tle_lines[2], tle_lines[0], ts)

# Geod untuk hitung area di ellipsoid WGS84
geod = Geod(ellps="WGS84")

# === Variabel CSV & Kontrol Penghentian ===
csv_filename = "spotbeam_data.csv"
save_counter = 0
max_saves = 5 # Batas maksimum baris CSV
stop_simulation = False # Variabel kontrol penghentian utama

# buat file baru jika belum ada
if not os.path.exists(csv_filename):
    with open(csv_filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["UTC_time", "Latitude", "Longitude", "Altitude_km", "Spotbeam_Area_km2"])

def latlon_at_time(time_ts):
    geocentric = sat.at(time_ts)
    subpoint = wgs84.subpoint(geocentric)
    return subpoint.latitude.degrees, subpoint.longitude.degrees, subpoint.elevation.km

# Setup figure
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.stock_img()
ax.coastlines(linewidth=0.6)

# Inisialisasi plot
line_artist, = ax.plot([], [], color='red', linewidth=2, label='Lintasan Satelit', transform=ccrs.Geodetic())
point_artist, = ax.plot([], [], 'bo', markersize=6, label='Satelit', transform=ccrs.Geodetic(), zorder=5)
spotbeam_artist, = ax.plot([], [], color='blue', alpha=0.3, transform=ccrs.Geodetic(), label="Spotbeam Strip")

time_text = ax.text(0.01, 0.03, '', transform=ax.transAxes, fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.7))

area_text = ax.text(0.01, 0.08, '', transform=ax.transAxes, fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.7))

plt.title('Animasi Satelit dengan Spotbeam Strip (Ground Track)')
plt.legend(loc='upper right')

# Variabel penyimpan lintasan
track_lats = []
track_lons = []

# Variabel kontrol crossing orbit
start_real_time = datetime.utcnow().replace(tzinfo=timezone.utc)
start_sim_time = start_real_time
first_lat, first_lon = None, None # Titik awal simulasi
start_point = None # Titik awal sebagai objek Shapely
last_cross_state = False
cross_tolerance = 5.0

# Variabel untuk luas area cakupan
spotbeam_union = None
last_area_km2 = 0.0

def update(frame):
    global track_lats, track_lons, first_lat, first_lon, start_point, last_cross_state
    global spotbeam_union, last_area_km2, save_counter, stop_simulation

    # 🛑 MEKANISME PENGHENTIAN
    if stop_simulation:
        # Hentikan proses update frame jika kondisi penghentian terpenuhi
        return (point_artist, line_artist, spotbeam_artist, time_text, area_text)

    elapsed_real = frame * (update_interval_ms / 1000.0)
    sim_seconds = elapsed_real * speedup
    sim_time = start_sim_time + timedelta(seconds=sim_seconds)
    t_ts = ts.utc(sim_time.year, sim_time.month, sim_time.day,
                  sim_time.hour, sim_time.minute, sim_time.second + sim_time.microsecond/1e6)

    lat, lon, alt = latlon_at_time(t_ts)

    # Normalisasi longitude ke -180..180
    if lon > 180:
        lon -= 360
    elif lon < -180:
        lon += 360

    # Simpan dan definisikan titik awal hanya sekali
    if first_lat is None and first_lon is None:
        first_lat, first_lon = lat, lon
        start_point = Point(first_lon, first_lat) # Definisikan titik awal Shapely

    # ... (Logika deteksi crossing dan pembersihan lintasan tetap sama) ...
    dist = np.sqrt((lat - first_lat)**2 + (lon - first_lon)**2)
    is_crossing = dist < cross_tolerance
    if is_crossing and not last_cross_state:
        # Logika ini untuk membersihkan lintasan visual, bukan penghenti CSV utama lagi
        track_lats, track_lons = [], []
        # first_lat, first_lon = lat, lon # TIDAK di-reset agar start_point tetap sama
    last_cross_state = is_crossing

    # Tambahkan ke lintasan
    track_lats.append(lat)
    track_lons.append(lon)

    # Update titik & garis lintasan
    point_artist.set_data([lon], [lat])
    line_artist.set_data(track_lons, track_lats)

    # Hitung strip coverage
    if len(track_lats) > 1:
        lon_prev = track_lons[-2]
        lat_prev = track_lats[-2]
        dx, dy = lon - lon_prev, lat - lat_prev
        length = np.hypot(dx, dy)
        if length > 0:
            dx, dy = dx/length, dy/length
            nx, ny = -dy, dx
            half_w = (spotbeam_width_km / 111.0) / 2
            current_strip = Polygon([
                (lon_prev + nx*half_w, lat_prev + ny*half_w),
                (lon_prev - nx*half_w, lat_prev - ny*half_w),
                (lon      - nx*half_w, lat      - ny*half_w),
                (lon      + nx*half_w, lat      + ny*half_w)
            ])
            
            if spotbeam_union is None:
                spotbeam_union = current_strip
            else:
                # Akumulasi area cakupan
                spotbeam_union = unary_union([spotbeam_union, current_strip])
            
            x, y = current_strip.exterior.xy
            spotbeam_artist.set_data(x, y)

            if spotbeam_union.is_valid and not spotbeam_union.is_empty:
                polys = [spotbeam_union] if spotbeam_union.geom_type == "Polygon" else list(spotbeam_union.geoms)
                total_area_m2 = 0
                for poly in polys:
                    if len(poly.exterior.coords) >= 4:
                        poly_x, poly_y = poly.exterior.xy
                        area_m2, _ = geod.polygon_area_perimeter(poly_x, poly_y)
                        total_area_m2 += abs(area_m2)
                last_area_km2 = total_area_m2 / 1e6
                
                # 🎯 PENGHENTIAN BARU: Cek apakah titik awal sudah tercakup
                if start_point is not None and spotbeam_union.contains(start_point):
                    if save_counter < max_saves:
                        # Simpan baris terakhir sebelum berhenti
                        with open(csv_filename, "a", newline="") as f:
                            writer = csv.writer(f)
                            writer.writerow([
                                sim_time.strftime('%Y-%m-%d %H:%M:%S'),
                                f"{lat:.6f}", f"{lon:.6f}", f"{alt:.2f}", f"{last_area_km2:.2f}"
                            ])
                        save_counter += 1
                        print(f"\n[INFO] Titik awal (lat={first_lat:.2f}, lon={first_lon:.2f}) TERCATAT di CSV (ke-{save_counter}) dan Simulasi Dihentikan oleh Coverage.")
                    else:
                         print(f"\n[INFO] Titik awal (lat={first_lat:.2f}, lon={first_lon:.2f}) masuk ke area spotbeam, tetapi batas CSV (5 data) sudah tercapai.")
                    
                    stop_simulation = True # Set flag untuk menghentikan animasi

    # Batasi jumlah titik lintasan (2 orbit)
    orbit_period_sec = 5520
    frame_sim_sec = (update_interval_ms/1000.0) * speedup
    max_points = int(2 * orbit_period_sec / frame_sim_sec)
    
    # 💾 PENYIMPANAN DATA PERIODE (Pengganti Mekanisme Lama)
    if len(track_lats) > max_points:
        # Jika belum dihentikan oleh kondisi coverage DAN belum mencapai batas baris
        if not stop_simulation and save_counter < max_saves:
            with open(csv_filename, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([
                    sim_time.strftime('%Y-%m-%d %H:%M:%S'),
                    f"{lat:.6f}", f"{lon:.6f}", f"{alt:.2f}", f"{last_area_km2:.2f}"
                ])
            save_counter += 1
            print(f"[INFO] Data spotbeam disimpan ke CSV (ke-{save_counter}) berdasarkan batas orbit.")
            
        # Logika membersihkan lintasan
        track_lats.pop(0)
        track_lons.pop(0)

    # Update teks
    area_text.set_text(f"Cakupan spotbeam: {last_area_km2/1e3:.2f} ribu km²")
    time_text.set_text(
        f"UTC sim: {sim_time.strftime('%Y-%m-%d %H:%M:%S')} | "
        f"lat={lat:.2f}, lon={lon:.2f}, alt={alt:.1f} km"
    )

    return (point_artist, line_artist, spotbeam_artist, time_text, area_text)

# Fungsi untuk menghentikan animasi Matplotlib (Wajib karena FuncAnimation tidak berhenti otomatis)
def check_stop():
    if stop_simulation:
        anim.event_source.stop()
        print("[INFO] Animasi Matplotlib Dihentikan.")
    else:
        fig.canvas.manager.window.after(100, check_stop)

anim = FuncAnimation(fig, update, interval=update_interval_ms, blit=True)

# Panggil pengecekan penghentian
if fig.canvas.manager:
    fig.canvas.manager.window.after(100, check_stop)

plt.show()

C:\Users\ADVAN\AppData\Local\Temp\ipykernel_17580\237285704.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_real_time = datetime.utcnow().replace(tzinfo=timezone.utc)
C:\Users\ADVAN\AppData\Local\Temp\ipykernel_17580\237285704.py:224: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  anim = FuncAnimation(fig, update, interval=update_interval_ms, blit=True)


FIX INSYAALLAH

NOTE : GA LIMIT 5, TAPI MALAH INFITE DAN DATA NYA BNYK BAT ANJAYY
GW RAGU DATA NYA BENER-BENER TIAP ORBIT

kyk nya ada data overleap deh, karena parameter nya msh ambigu


In [11]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.animation import FuncAnimation
from skyfield.api import EarthSatellite, load, wgs84
from datetime import datetime, timedelta, timezone
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union
from pyproj import Geod
import csv, os

# ---------- TLE ISS (ZARYA) ----------
tle_lines = [
    "EXPLORER 22",
    "1 00899U 64064A   25276.49600160  .00000579  00000-0  49569-3 0  9991",
    "2 00899  79.6909  47.8810 0120383 146.1380 214.7533 13.82947257 69497"
]
# -------------------------------------

update_interval_ms = 200     # ms per frame
speedup = 127                # percepatan waktu simulasi
spotbeam_width_km = 2000     # lebar strip coverage tegak lurus lintasan (km)

# Inisialisasi skyfield
ts = load.timescale()
sat = EarthSatellite(tle_lines[1], tle_lines[2], tle_lines[0], ts)

# Geod untuk hitung area di ellipsoid WGS84
geod = Geod(ellps="WGS84")

# === Variabel CSV & Kontrol Penghentian ===
csv_filename = "spotbeam_data.csv"
save_counter = 0 # Tetap digunakan untuk penomoran INFO
# max_saves = 5 <--- DIHAPUS
stop_simulation = False # Variabel kontrol penghentian utama

# buat file baru jika belum ada
if not os.path.exists(csv_filename):
    with open(csv_filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["UTC_time", "Latitude", "Longitude", "Altitude_km", "Spotbeam_Area_km2"])

def latlon_at_time(time_ts):
    geocentric = sat.at(time_ts)
    subpoint = wgs84.subpoint(geocentric)
    return subpoint.latitude.degrees, subpoint.longitude.degrees, subpoint.elevation.km

# Setup figure
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.stock_img()
ax.coastlines(linewidth=0.6)

# Inisialisasi plot
line_artist, = ax.plot([], [], color='red', linewidth=2, label='Lintasan Satelit', transform=ccrs.Geodetic())
point_artist, = ax.plot([], [], 'bo', markersize=6, label='Satelit', transform=ccrs.Geodetic(), zorder=5)
spotbeam_artist, = ax.plot([], [], color='blue', alpha=0.3, transform=ccrs.Geodetic(), label="Spotbeam Strip")

time_text = ax.text(0.01, 0.03, '', transform=ax.transAxes, fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.7))

area_text = ax.text(0.01, 0.08, '', transform=ax.transAxes, fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.7))

plt.title('Animasi Satelit dengan Spotbeam Strip (Ground Track)')
plt.legend(loc='upper right')

# Variabel penyimpan lintasan
track_lats = []
track_lons = []

# Variabel kontrol crossing orbit
start_real_time = datetime.utcnow().replace(tzinfo=timezone.utc)
start_sim_time = start_real_time
first_lat, first_lon = None, None
start_point = None # Titik awal sebagai objek Shapely
last_cross_state = False
cross_tolerance = 5.0

# Variabel untuk luas area cakupan
spotbeam_union = None
last_area_km2 = 0.0

def update(frame):
    global track_lats, track_lons, first_lat, first_lon, start_point, last_cross_state
    global spotbeam_union, last_area_km2, save_counter, stop_simulation

    # 🛑 MEKANISME PENGHENTIAN: Hentikan proses jika bendera diaktifkan
    if stop_simulation:
        return (point_artist, line_artist, spotbeam_artist, time_text, area_text)

    elapsed_real = frame * (update_interval_ms / 1000.0)
    sim_seconds = elapsed_real * speedup
    sim_time = start_sim_time + timedelta(seconds=sim_seconds)
    t_ts = ts.utc(sim_time.year, sim_time.month, sim_time.day,
                  sim_time.hour, sim_time.minute, sim_time.second + sim_time.microsecond/1e6)

    lat, lon, alt = latlon_at_time(t_ts)

    # Normalisasi longitude ke -180..180
    if lon > 180:
        lon -= 360
    elif lon < -180:
        lon += 360

    # Simpan dan definisikan titik awal hanya sekali
    if first_lat is None and first_lon is None:
        first_lat, first_lon = lat, lon
        start_point = Point(first_lon, first_lat)

    # ... (Logika deteksi crossing dan pembersihan lintasan tetap sama) ...
    dist = np.sqrt((lat - first_lat)**2 + (lon - first_lon)**2)
    is_crossing = dist < cross_tolerance
    if is_crossing and not last_cross_state:
        # Logika pembersihan lintasan visual
        track_lats, track_lons = [], []
    last_cross_state = is_crossing

    # Tambahkan ke lintasan
    track_lats.append(lat)
    track_lons.append(lon)

    # Update titik & garis lintasan
    point_artist.set_data([lon], [lat])
    line_artist.set_data(track_lons, track_lats)

    # Hitung strip coverage
    if len(track_lats) > 1:
        lon_prev = track_lons[-2]
        lat_prev = track_lats[-2]
        dx, dy = lon - lon_prev, lat - lat_prev
        length = np.hypot(dx, dy)
        if length > 0:
            dx, dy = dx/length, dy/length
            nx, ny = -dy, dx
            half_w = (spotbeam_width_km / 111.0) / 2
            current_strip = Polygon([
                (lon_prev + nx*half_w, lat_prev + ny*half_w),
                (lon_prev - nx*half_w, lat_prev - ny*half_w),
                (lon      - nx*half_w, lat      - ny*half_w),
                (lon      + nx*half_w, lat      + ny*half_w)
            ])
            
            if spotbeam_union is None:
                spotbeam_union = current_strip
            else:
                # Akumulasi area cakupan
                spotbeam_union = unary_union([spotbeam_union, current_strip])
            
            x, y = current_strip.exterior.xy
            spotbeam_artist.set_data(x, y)

            if spotbeam_union.is_valid and not spotbeam_union.is_empty:
                polys = [spotbeam_union] if spotbeam_union.geom_type == "Polygon" else list(spotbeam_union.geoms)
                total_area_m2 = 0
                for poly in polys:
                    if len(poly.exterior.coords) >= 4:
                        poly_x, poly_y = poly.exterior.xy
                        area_m2, _ = geod.polygon_area_perimeter(poly_x, poly_y)
                        total_area_m2 += abs(area_m2)
                last_area_km2 = total_area_m2 / 1e6
                
                # 🎯 PENGHENTIAN UTAMA: Cek apakah titik awal sudah tercakup
                if start_point is not None and spotbeam_union.contains(start_point):
                    # Simpan baris terakhir sebelum berhenti
                    with open(csv_filename, "a", newline="") as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            sim_time.strftime('%Y-%m-%d %H:%M:%S'),
                            f"{lat:.6f}", f"{lon:.6f}", f"{alt:.2f}", f"{last_area_km2:.2f}"
                        ])
                    save_counter += 1
                    print(f"\n[INFO] Titik awal (lat={first_lat:.2f}, lon={first_lon:.2f}) TERCATAT di CSV (ke-{save_counter}) dan Simulasi Dihentikan oleh Coverage.")
                    
                    stop_simulation = True # Set flag untuk menghentikan animasi
                    return (point_artist, line_artist, spotbeam_artist, time_text, area_text) # Keluar dari update

    # Batasi jumlah titik lintasan (2 orbit)
    orbit_period_sec = 5520
    frame_sim_sec = (update_interval_ms/1000.0) * speedup
    max_points = int(2 * orbit_period_sec / frame_sim_sec)
    
    # 💾 PENYIMPANAN DATA PERIODE (Berlanjut Tanpa Batas Maksimum Baris)
    if len(track_lats) > max_points:
        # Jika belum dihentikan oleh kondisi coverage, simpan data orbit penuh
        if not stop_simulation:
            with open(csv_filename, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([
                    sim_time.strftime('%Y-%m-%d %H:%M:%S'),
                    f"{lat:.6f}", f"{lon:.6f}", f"{alt:.2f}", f"{last_area_km2:.2f}"
                ])
            save_counter += 1
            print(f"[INFO] Data spotbeam disimpan ke CSV (ke-{save_counter}) berdasarkan batas orbit.")
            
        # Logika membersihkan lintasan
        track_lats.pop(0)
        track_lons.pop(0)

    # Update teks
    area_text.set_text(f"Cakupan spotbeam: {last_area_km2/1e3:.2f} ribu km²")
    time_text.set_text(
        f"UTC sim: {sim_time.strftime('%Y-%m-%d %H:%M:%S')} | "
        f"lat={lat:.2f}, lon={lon:.2f}, alt={alt:.1f} km"
    )

    return (point_artist, line_artist, spotbeam_artist, time_text, area_text)

# Fungsi untuk menghentikan animasi Matplotlib (Wajib karena FuncAnimation tidak berhenti otomatis)
def check_stop():
    if stop_simulation:
        anim.event_source.stop()
        print("[INFO] Animasi Matplotlib Dihentikan.")
    else:
        fig.canvas.manager.window.after(100, check_stop)

anim = FuncAnimation(fig, update, interval=update_interval_ms, blit=True)

# Panggil pengecekan penghentian
if fig.canvas.manager:
    fig.canvas.manager.window.after(100, check_stop)

plt.show()

C:\Users\TEMANS\AppData\Local\Temp\ipykernel_7484\3812179567.py:220: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  anim = FuncAnimation(fig, update, interval=update_interval_ms, blit=True)


[INFO] Data spotbeam disimpan ke CSV (ke-1) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-2) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-3) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-4) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-5) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-6) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-7) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-8) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-9) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-10) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-11) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-12) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-13) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan ke CSV (ke-14) berdasarkan batas orbit.
[INFO] Data spotbeam disimpan

In [2]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.animation import FuncAnimation
from skyfield.api import EarthSatellite, load, wgs84
from datetime import datetime, timedelta, timezone
from shapely.geometry import Polygon
from shapely.ops import unary_union
from pyproj import Geod
import csv, os

# ---------- TLE ISS (ZARYA) ----------
tle_lines = [
    "ISS (ZARYA)",
    "1 09022U 76069A   25276.66381223  .00000555  00000-0  19691-3 0  9998",
    "2 09022  74.0437  52.3216 0013176 202.6488 157.4086 14.36709130575537"
]
# -------------------------------------

update_interval_ms = 200
speedup = 127
spotbeam_width_km = 2000

# Inisialisasi skyfield
ts = load.timescale()
sat = EarthSatellite(tle_lines[1], tle_lines[2], tle_lines[0], ts)

# Ambil epoch pertama dari TLE
epoch_datetime = sat.epoch.utc_datetime().replace(tzinfo=timezone.utc)
print("[INFO] Epoch TLE:", epoch_datetime)

# Geod untuk hitung area
geod = Geod(ellps="WGS84")

# === Variabel CSV ===
csv_filename = "spotbeam_data.csv"
save_enabled = 0   # akan jadi False setelah epoch tercapai

if not os.path.exists(csv_filename):
    with open(csv_filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["UTC_time", "Latitude", "Longitude", "Altitude_km", "Spotbeam_Area_km2"])

def latlon_at_time(time_ts):
    geocentric = sat.at(time_ts)
    subpoint = wgs84.subpoint(geocentric)
    return subpoint.latitude.degrees, subpoint.longitude.degrees, subpoint.elevation.km

# Setup figure
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.stock_img()
ax.coastlines(linewidth=0.6)

line_artist, = ax.plot([], [], color='red', linewidth=2, label='Lintasan Satelit', transform=ccrs.Geodetic())
point_artist, = ax.plot([], [], 'bo', markersize=6, label='Satelit', transform=ccrs.Geodetic(), zorder=5)
spotbeam_artist, = ax.plot([], [], color='blue', alpha=0.3, transform=ccrs.Geodetic(), label="Spotbeam Strip")

time_text = ax.text(0.01, 0.03, '', transform=ax.transAxes, fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.7))
area_text = ax.text(0.01, 0.08, '', transform=ax.transAxes, fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.7))

plt.title('Animasi Satelit dengan Spotbeam Strip (Ground Track)')
plt.legend(loc='upper right')

track_lats, track_lons = [], []
start_real_time = datetime.utcnow().replace(tzinfo=timezone.utc)
start_sim_time = start_real_time
first_lat, first_lon = None, None
last_cross_state = False
cross_tolerance = 5.0

spotbeam_union = None
last_area_km2 = 0.0

def update(frame):
    global track_lats, track_lons, first_lat, first_lon, last_cross_state
    global spotbeam_union, last_area_km2, save_enabled

    elapsed_real = frame * (update_interval_ms / 1000.0)
    sim_seconds = elapsed_real * speedup
    sim_time = start_sim_time + timedelta(seconds=sim_seconds)
    t_ts = ts.utc(sim_time.year, sim_time.month, sim_time.day,
                  sim_time.hour, sim_time.minute, sim_time.second + sim_time.microsecond/1e6)

    lat, lon, alt = latlon_at_time(t_ts)
    if lon > 180: lon -= 360
    elif lon < -180: lon += 360

    if first_lat is None and first_lon is None:
        first_lat, first_lon = lat, lon

    dist = np.sqrt((lat - first_lat)**2 + (lon - first_lon)**2)
    is_crossing = dist < cross_tolerance
    if is_crossing and not last_cross_state:
        spotbeam_union = None
        track_lats, track_lons = [], []
        first_lat, first_lon = lat, lon
    last_cross_state = is_crossing

    track_lats.append(lat)
    track_lons.append(lon)

    point_artist.set_data([lon], [lat])
    line_artist.set_data(track_lons, track_lats)

    if len(track_lats) > 1:
        lon_prev, lat_prev = track_lons[-2], track_lats[-2]
        dx, dy = lon - lon_prev, lat - lat_prev
        length = np.hypot(dx, dy)
        if length > 0:
            dx, dy = dx/length, dy/length
            nx, ny = -dy, dx
            half_w = (spotbeam_width_km / 111.0) / 2
            rect = Polygon([
                (lon_prev + nx*half_w, lat_prev + ny*half_w),
                (lon_prev - nx*half_w, lat_prev - ny*half_w),
                (lon     - nx*half_w, lat     - ny*half_w),
                (lon     + nx*half_w, lat     + ny*half_w)
            ])
            if spotbeam_union is None:
                spotbeam_union = rect
            else:
                spotbeam_union = unary_union([spotbeam_union, rect])
            x, y = rect.exterior.xy
            spotbeam_artist.set_data(x, y)

            if spotbeam_union.is_valid and not spotbeam_union.is_empty:
                polys = [spotbeam_union] if spotbeam_union.geom_type == "Polygon" else list(spotbeam_union.geoms)
                total_area_m2 = 0
                for poly in polys:
                    poly_x, poly_y = poly.exterior.xy
                    area_m2, _ = geod.polygon_area_perimeter(poly_x, poly_y)
                    total_area_m2 += abs(area_m2)
                last_area_km2 = total_area_m2 / 1e6

    # Batasi jumlah titik lintasan (2 orbit)
    orbit_period_sec = 5520
    frame_sim_sec = (update_interval_ms/1000.0) * speedup
    max_points = int(2 * orbit_period_sec / frame_sim_sec)
    if len(track_lats) > max_points:
        track_lats.pop(0)
        track_lons.pop(0)

        # === Simpan ke CSV sampai epoch tercapai ===
        if sim_time < epoch_datetime:
            with open(csv_filename, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([
                    sim_time.strftime('%Y-%m-%d %H:%M:%S'),
                    f"{lat:.6f}", f"{lon:.6f}", f"{alt:.2f}", f"{last_area_km2:.2f}"
                ])
            save_enabled+= 1
            print(f"[INFO] Data spotbeam disimpan ke CSV @ {sim_time}")
        elif  sim_time >= epoch_datetime:
            print("[STOP] Penyimpanan CSV dihentikan (sampai epoch pertama)")

    area_text.set_text(f"Cakupan spotbeam: {last_area_km2/1e3:.2f} ribu km²")
    time_text.set_text(
        f"UTC sim: {sim_time.strftime('%Y-%m-%d %H:%M:%S')} | "
        f"lat={lat:.2f}, lon={lon:.2f}, alt={alt:.1f} km"
    )

    return (point_artist, line_artist, spotbeam_artist, time_text, area_text)

anim = FuncAnimation(fig, update, interval=update_interval_ms, blit=True)
plt.show()


[INFO] Epoch TLE: 2025-10-03 15:55:53.376648+00:00


C:\Users\TEMANS\AppData\Local\Temp\ipykernel_12752\550698991.py:170: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  anim = FuncAnimation(fig, update, interval=update_interval_ms, blit=True)
Traceback (most recent call last):
  File "C:\Users\TEMANS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "C:\Users\TEMANS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\matplotlib\animation.py", line 912, in _start
    self._init_draw()
  File "C:\Users\TEMANS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2

In [15]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from skyfield.api import EarthSatellite, load, wgs84
from datetime import datetime, timedelta, timezone
import csv
import os

# === Muat TLE ===
tle_lines = [
    "ISS (ZARYA)",
    "1 25544U 98067A   24300.51167824  .00010210  00000+0  19731-3 0  9994",
    "2 25544  51.6436  62.4935 0004316  94.5114  28.9810 15.50044754426033"
]

sat = EarthSatellite(tle_lines[1], tle_lines[2], tle_lines[0])
ts = load.timescale()

# === Epoch Awal dan Batas Akhir ===
start_time = datetime.now(timezone.utc)
end_time = start_time + timedelta(minutes=120)  # simulasi 2 jam

times = []
lats = []
lons = []

# === Simulasi Pergerakan Satelit ===
t = start_time
step = timedelta(seconds=10)  # setiap 10 detik

print(f"Epoch pertama satelit: {sat.epoch.utc_datetime()}")

while t <= end_time:
    time_ts = ts.utc(t.year, t.month, t.day, t.hour, t.minute, t.second)
    subpoint = sat.at(time_ts).subpoint()
    lat, lon = subpoint.latitude.degrees, subpoint.longitude.degrees

    times.append(t.isoformat())
    lats.append(lat)
    lons.append(lon)

    # Jika satelit menabrak epoch pertama, hentikan loop
    if abs((t - sat.epoch.utc_datetime()).total_seconds()) < 5:
        print(f"Berhenti di waktu {t} karena mendekati epoch pertama.")
        break

    t += step

print(f"Jumlah data yang dikumpulkan: {len(times)}")

# === Simpan ke CSV ===
if len(times) > 0:
    csv_path = os.path.join(os.getcwd(), "satellite_track.csv")
    with open(csv_path, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["time", "latitude", "longitude"])
        for i in range(len(times)):
            writer.writerow([times[i], lats[i], lons[i]])
    print(f"✅ Data tersimpan ke {csv_path}")
else:
    print("⚠️ Tidak ada data yang dikumpulkan — file tidak dibuat.")


Epoch pertama satelit: 2024-10-26 12:16:48.999908+00:00
Jumlah data yang dikumpulkan: 721
✅ Data tersimpan ke c:\Users\TEMANS\Documents\Zoom\satellite_track.csv
